## <span style="color:#a4d4a3">**Recursive Bayes Filter**</span>

The Recursive Bayes Filter provides a framework for recursive state estimation. It estimates the <span style="color:#ffa500">**state**</span> $ x $ of a system given <span style="color:#ffa500">**observations**</span> $ z $ and <span style="color:#ffa500">**controls**</span> $ u $:

$$ p(x \mid z, u) $$

---



#### 💭 <span style="color:#a4d4a3">Definition of the Belief</span>

The belief represents the <span style="color:#ffa500">**posterior**</span> distribution of the current state $ x_t $ given all observations $ z_{1:t} $ and control inputs $ u_{1:t}m $, written as:

$$
bel(x_t) = p(x_t \mid z_{1:t}, u_{1:t}) \quad\quad (1)
$$

In order to update the belief based on new measurements and controls we apply Bayes' Rule.

<details>
<summary>📝 <strong><span style="color:#e74c3c">Note:</span> </strong> <em> What does the state vector contain? </em> </summary>
<br>

The composition of the state vector $ x $ depends on the specific estimation problem we aim to solve:
- **Localization Only**:  

  For **2D localization**, the state vector could include the robot’s position and orientation:
  $$ x = [x, y, \theta]^T $$
  For **3D localization**, the state vector could include the robot's 3D position and orientation:
  $$ x = [x, y, z, \phi, \theta, \psi]^T $$

- **Mapping Only**:  

  In a scenario where the robot's pose is known and the goal is to estimate the map (for example, landmark positions), the state vector could consist of coordinates of landmarks in the environment. For $ n $ landmarks in **2D**, the state vector could be:
  $$ x = [l_{1,x}, l_{1,y}, l_{2,x}, l_{2,y}, \dots, l_{n,x}, l_{n,y}]^T $$

- **Simultaneous Localization and Mapping (SLAM)**:  

  In SLAM, the state vector simultaneously estimates the robot's pose and the map's landmark positions. For **2D SLAM** with $ n $ landmarks, the state vector is typically structured as:

  $$
  x = [x, y, \theta, l_{1,x}, l_{1,y}, l_{2,x}, l_{2,y}, \dots, l_{n,x}, l_{n,y}]^T
  $$

</details>

#### ✏️ <span style="color:#a4d4a3">Mathematical Derivation</span>

Using Bayes' Rule, we expand the belief in Eq. (1) as follows:

$$
\begin{aligned}
bel(x_t) &= p(x_t \mid z_{1:t}, u_{1:t}) \\
&= \eta \cdot \underbrace{p(z_t \mid x_t, z_{1:t-1}, u_{1:t})}_{likelihood} \cdot \underbrace{p(x_t \mid z_{1:t-1}, u_{1:t})}_{prior} \quad\quad (2)
\end{aligned}
$$

where:

- **$\eta = \frac{1}{p(z_t \mid z_{1:t-1}, u_{1:t})}$** is a constant, independent of the current state $x_t$. It serves as the normalizer (also known as the <span style="color:#ffa500">**evidence**</span>) for the new observation $z_t$, ensuring the resulting posterior distribution integrates to 1.

- **$p(x_t \mid z_{1:t-1}, u_{1:t})$** is the <span style="color:#ffa500">**prior**</span>, representing our predicted belief about the current state before incorporating the latest measurement. This prior comes from the **motion model** and is based on past observations and all known control inputs.

- **$p(z_t \mid x_t, z_{1:t-1}, u_{1:t})$** is the <span style="color:#ffa500">**likelihood**</span> obtained from the <span style="color:#ffa500">**observation model**</span>. It quantifies the probability of receiving the current measurement $z_t$, given the current state of the world $x_t$, past observations, and all known control inputs.

<details>
<summary>📝 <strong><span style="color:#e74c3c">Note:</span> </strong> <em> How did we apply Bayes' Rule? </em> </summary>
<br>

- <span style="color:#ffa500">**Bayes' Rule**</span> is given by:  
$$
p(A \mid B) = \frac{p(B \mid A) \cdot p(A)}{p(B)}
$$

- The sequence of control inputs $u_{1:t}$ is given (known), and thus we condition on these control inputs as known parameters, not as observations.

- The previous observations $z_{1:t-1}$ are also known (fixed events), as they represent past sensor measurements.

- Therefore, we explicitly include both in our posterior probability:  
  $$
  p(x_t \mid z_t, z_{1:t-1}, u_{1:t})
  $$

- When multiple events are involved, we use the following form of Bayes' Rule:  
  $$
  p(A \mid B, C) = \frac{p(B \mid A, C) \cdot p(A \mid C)}{p(B \mid C)}
  $$

- By setting our events as follows:  
  - $A = x_t$ (current state),  
  - $B = z_t$ (current measurement),  
  - $C = z_{1:t-1}, u_{1:t}$ (fixed past measurements and known controls),  

  we have:  
  $$
  p(x_t \mid z_t, z_{1:t-1}, u_{1:t}) = \frac{p(z_t \mid x_t, z_{1:t-1}, u_{1:t}) \cdot p(x_t \mid z_{1:t-1}, u_{1:t})}{p(z_t \mid z_{1:t-1}, u_{1:t})}
  $$

This expression shows explicitly how we integrate new observations $z_t$ into our previous belief (prediction), resulting in an updated posterior belief of the current state $x_t$.

</details>

Continuing from Eq. (2), we introduce a key simplifying assumption known as the <span style="color:#ffa500">**Markov assumption**</span>:

> The current measurement $z_t$ is conditionally independent of past measurements and control commands given the current state $x_t$.  
>
> In other words, once the current state of the world $x_t$ is known, past observations $z_{1:t-1}$ and past control inputs $u_{1:t}$ do not provide
> any additional information when predicting the current measurement $z_t$.

Mathematically, this assumption simplifies to:

$$
p(z_t \mid x_t, z_{1:t-1}, u_{1:t}) = p(z_t \mid x_t)
$$

With this simplification, our belief update becomes:

$$
\begin{aligned}
bel(x_t) &= p(x_t \mid z_{1:t}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t, z_{1:t-1}, u_{1:t}) \cdot p(x_t \mid z_{1:t-1}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t) \cdot p(x_t \mid z_{1:t-1}, u_{1:t}) \quad\quad\quad\quad\quad\quad\quad (3)
\end{aligned}
$$

By applying the <span style="color:#ffa500">**law of total probability**</span> to Eq. (3), we obtain:

$$
\begin{aligned}
bel(x_t) &= p(x_t \mid z_{1:t}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t, z_{1:t-1}, u_{1:t}) \cdot p(x_t \mid z_{1:t-1}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t) \cdot p(x_t \mid z_{1:t-1}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t) \cdot \int_{x_{t-1}} p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t}) \cdot p(x_{t-1} \mid z_{1:t-1}, u_{1:t}) \, dx_{t-1} \quad\quad (4)
\end{aligned}
$$

<details>
<summary>📝 <strong><span style="color:#e74c3c">Note:</span> </strong> <em> Law of Total Probability (Reminder) </em> </summary>
<br>

The <span style="color:#ffa500">**law of total probability**</span> states that the probability of an event $A$ can be found by considering all possible events $B$ that cover the entire sample space. Mathematically, it's expressed as:

$$
p(A) = \int_{B} p(A \mid B) \cdot p(B) \, dB
$$

This means the probability distribution of event $A$ is obtained by integrating (or summing) the conditional probability of $A$ given $B$ multiplied by the probability of $B$, over all possible outcomes of $B$.

</details>

Moving forward, we introduce two additional simplifying <span style="color:#ffa500">**Markov assumptions**</span>:

1. **State transition assumption**:
   $$
   p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t}) = p(x_t \mid x_{t-1}, u_t)
   $$  
   This assumption implies that the current state depends only on the previous state and the most recent control input. In other words, knowing the previous state $x_{t-1}$ and the control command $u_t$ is sufficient to predict how the robot moves.

2. **Control independence assumption**:
   $$
   p(x_{t-1} \mid z_{1:t-1}, u_{1:t}) = p(x_{t-1} \mid z_{1:t-1}, u_{1:t-1})
   $$  
   This assumption states that the current state does not depend on future control commands. A future command does not provide any additional information about the past or current state.

With these assumptions, we can simplify our belief equation to the final recursive form:

$$
\begin{aligned}
bel(x_t) &= p(x_t \mid z_{1:t}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t, z_{1:t-1}, u_{1:t}) \cdot p(x_t \mid z_{1:t-1}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t) \cdot p(x_t \mid z_{1:t-1}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t) \cdot \int_{x_{t-1}} p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t}) \cdot p(x_{t-1} \mid z_{1:t-1}, u_{1:t}) \, dx_{t-1} \\
&= \eta \cdot p(z_t \mid x_t) \cdot \int_{x_{t-1}} p(x_t \mid x_{t-1}, u_t) \cdot p(x_{t-1} \mid z_{1:t-1}, u_{1:t-1}) \, dx_{t-1} \quad\quad (5)
\end{aligned}
$$

We recognize that the last term $p(x_{t-1} \mid z_{1:t-1}, u_{1:t-1})$ is precisely our previous belief $bel(x_{t-1})$. This is the recursive component of the Bayes filter, which allows us to continuously update our belief based on new measurements and controls. Thus, the final concise recursive equation becomes:

$$
\begin{aligned}
bel(x_t) &= p(x_t \mid z_{1:t}, u_{1:t}) \\
&= \eta \cdot p(z_t \mid x_t) \cdot \int_{x_{t-1}} p(x_t \mid x_{t-1}, u_t) \cdot bel(x_{t-1}) \, dx_{t-1} \quad\quad (6)
\end{aligned}
$$

---


### <span style="color:#a4d4a3">**Prediction and Correction Steps**</span>

The Recursive Bayes Filter can be represented as a <span style="color:#ffa500">**two-step recursive process**</span>:

#### 🔮 <span style="color:#a4d4a3">Prediction Step (**Motion Model**)</span>

In the prediction step, we estimate our new belief $ bel(\overline{x}_t) $ based on the previous belief and the control input (motion):

$$
bel(\overline{x}_t) = \int p(x_t \mid x_{t-1}, u_t) \, bel(x_{t-1}) \, dx_{t-1}
$$

- This step provides a prediction based on control commands (motion).
- $ bel(\overline{x}_t) $ represents the predicted belief <span style="color:#ffa500">**before**</span> incorporating sensor measurements.

#### 🛠️ <span style="color:#a4d4a3">Correction Step (**Observation Model**)</span>

In the correction step, we incorporate the latest sensor measurement $ z_t $ to correct our prediction from the previous step, resulting in an updated belief:

$$
bel(x_t) = \eta \cdot p(z_t \mid x_t) \cdot bel(\overline{x}_t)
$$

- This step adjusts the prediction using actual sensor data, providing a more accurate state estimate.

#### ✔️ <span style="color:#a4d4a3">Summary</span>

- The <span style="color:#00703c">**Bayes Filter**</span> is a general probabilistic framework for recursive state estimation. It consists of two essential steps:
  - <span style="color:#00703c">**Prediction Step:**</span> Estimate the current state based on a motion model and previous state information.
  - <span style="color:#00703c">**Correction Step:**</span> Refine this prediction using the most recent sensor observations.

- However, the Bayes Filter itself does not specify:
  - How exactly the integrals or probability distributions should be computed.
  - What assumptions to make regarding the motion and observation models, or how to handle noise in these models.

- Practical implementations (specific realizations) of the Bayes Filter include:
  - <span style="color:#00703c">**Kalman Filter (KF)**</span> for linear systems with Gaussian noise.
  - <span style="color:#00703c">**Extended Kalman Filter (EKF)**</span> for nonlinear systems using linear approximations.
  - <span style="color:#00703c">**Particle Filter (PF)**</span> for nonlinear systems with non-Gaussian noise or multimodal distributions.

Each of these filters represents a specific solution tailored to the unique characteristics of the system dynamics, sensor models, and noise properties.

---

### 📚 <span style="color:#a4d4a3">**Reading Material**</span>

**On the Bayes Filter**
- Thrun et al.: *"Probabilistic Robotics"*, **Chapter 2**